In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
url = 'https://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=10&vw=cf&SecurityToken=0P000003MC%5D3%5D0%5DE0WWE%24%24ALL&Id=0P000003MC&ClientFund=0&CurrencyId=BAS'


In [4]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)


In [5]:
#Store contents of the website under doc
doc = lh.fromstring(page.content)


In [6]:
#Parse data stored between <tr>
tr_elements = doc.xpath('//tr')
tr_elements.pop(0)

<Element tr at 0x1410e73d0e8>

In [7]:
#Check length of first 12 rows
[len(T) for T in tr_elements[:20]]

[6, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 6, 6, 6, 6, 6, 6, 6, 1]

In [8]:
#Parse first row as header (financial years)
#Create empty list
col=[]
i=0

#For each row, store each first elements (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:""
2:"2015"
3:"2016"
4:"2017"
5:"2018"
6:"2019"


In [9]:
#Since first row is header, data stored on second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #if row is not of size 6, the //tr data is not from our table (should just delete primary headings but retain line items)
    if len(T)!=6:
        continue
        
    #i is the index of our column
    i=0
    
    #iterate through each element of row
    for t in T.iterchildren():
        data = t.text_content()
        #check if row is empty
        if i>0:
            try:
                data=int(data)
            except:
                pass
        #Append data to empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        

In [10]:
#check column lengths
[len(C) for (title,C) in col]

[26, 26, 26, 26, 26, 26]

In [12]:
#Create DataFrame
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

df.head()

,,2015,2016,2017,2018,2019
0,Net Income,-,-,-,-,-
1,Depreciation and amortization,"2,667.00","2,980.00","3,861.00","4,759.00","5,424.00"
2,Deferred income taxes,-,-,-,-,-
3,Accounts receivable,393.00,465.00,"-1,651.00","-1,734.00","2,431.00"
4,Inventory,116.00,-549.00,50.00,-472.00,"-1,528.00"
